<a href="https://colab.research.google.com/github/yb2462/rsLQR/blob/main/Copy_of_Cholesky.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Installing CUDA

In [ ]:
#installing CUDA
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [ ]:
#GPU runtime
!nvidia-smi

Wed Jun  7 03:42:39 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   69C    P8    12W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# formatting helpers
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-63_mqqty
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-63_mqqty
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4287 sha256=1d815742ff0a1c2350e11e29d5cc8ca1f9f5fd5b2f34996740511ebba1af3c19
  Stored in directory: /tmp/pip-ephem-wheel-cache-yajjvzzy/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built NVCCPlugin
created output directory at /content/src
Out bin /content/result.out


In [ ]:
!wget -O cpp_plugin.py https://gist.github.com/akshaykhadse/7acc91dd41f52944c6150754e5530c4b/raw/cpp_plugin.py


--2023-06-07 03:42:46--  https://gist.github.com/akshaykhadse/7acc91dd41f52944c6150754e5530c4b/raw/cpp_plugin.py
Resolving gist.github.com (gist.github.com)... 192.30.255.112
Connecting to gist.github.com (gist.github.com)|192.30.255.112|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://gist.githubusercontent.com/akshaykhadse/7acc91dd41f52944c6150754e5530c4b/raw/cpp_plugin.py [following]
--2023-06-07 03:42:46--  https://gist.githubusercontent.com/akshaykhadse/7acc91dd41f52944c6150754e5530c4b/raw/cpp_plugin.py
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2730 (2.7K) [text/plain]
Saving to: ‘cpp_plugin.py’

cpp_plugin.py       100%[===================>]   2.67K  --.-KB/s    in 0s      

2023-06-07 03:42:4

In [ ]:
%load_ext cpp_plugin


# Cholesky Factorization - done and tested, should we pass n as well+should we put 0s?

**Cholesky Factorization** 

$A = L L^T$ 
where L is lower triangular matrix 
or 

$A = U^T U$ where U is an upper triangular matrix.

We will implement the lower triangular factorization as it was implemented in the original rsLQR paper.

*Check [link text](https://github.com/A2R-Lab/Block-Factorization/blob/main/SRI-23/choleskyDecomp_InPlace.cu)
for column major order implementation +cgrps


In [ ]:
%%cuda -n run.cu

#include <stdio.h>
#include <iostream>
#include <cmath>


/**
 * @brief Perform a Cholesky decomposition in place
 *
 * Performs a Cholesky decomposition on the square matrix @p s_A, storing the result in the
 * lower triangular portion of @p s_A.
 *
 * @param  s_A a square symmetric matrix 
 * @return clap_kCholeskySuccess if successful, and clap_kCholeskyFail if not.
 */
template <typename T> 
__device__ 
void cholDecomp_InPlace(T *s_A, int n) {
    for (unsigned col = 0; col < n; col++) {
        if (threadIdx.x == 0){
            T sum = 0;
            T val = s_A[n*col+col]; //entry Ljj
            for(unsigned col_l = 0 ; col_l < col; col_l++) {
                sum += pow(s_A[col*n+col_l],2);
            }
            s_A[col*n+col] = sqrt(val - sum);

        }
        __syncthreads(); //here we computed the diagonal entry of the column
        // compute the rest of the column
        for(unsigned row = threadIdx.x + col +1; row < n; row += blockDim.x){
            T sum = 0;
            T val = s_A[row*n+col];
            for(unsigned k = 0; k < col; k++) {
                sum += s_A[row*n+k]*s_A[col*n+k];
            }
            s_A[row*n+col] = (1.0/s_A[col*n+col])*(s_A[row*n+col]-sum);
        }
        __syncthreads();
    }
}

template <typename T> //can put 9 as a template variable, or dynamically allocate array
__global__ 
void choleskyDecomposition_InPlace_Kernel(T *d_A, int n) {
    __shared__ T s_A[n*n];
    // move RAM to shared
    for(unsigned i = threadIdx.x; i < n*n; i += blockDim.x){s_A[i] = d_A[i]; }

    // compute the decomposition
    cholDecomp_InPlace<T>(s_A, n);

    // move shared to RAM
    for(unsigned i = threadIdx.x; i < n*n; i += blockDim.x){d_A[i] = s_A[i];}
}



__host__
int main() {
    // Input matrix on the host
    int n=3;
    float A[n*n] = {6,15,55,15,55,225,55,225,979};


    // Allocate memory on the GPU for the input and output matrices
    float* d_A; cudaMalloc((void**)&d_A, n * n * sizeof(float));

    // Copy the input matrix from the host to the GPU memory
    cudaMemcpy(d_A, A, n * n * sizeof(float), cudaMemcpyHostToDevice);

    //Launch the CUDA kernel with appropriate block and grid dimensions
    int blockSize = 256;
    int gridSize = 1;
    choleskyDecomposition_InPlace_Kernel<float><<<gridSize, blockSize>>>(d_A, n);
    cudaDeviceSynchronize();
    // Copy the result back from the GPU memory to the host
    cudaMemcpy(A, d_A, n * n * sizeof(float), cudaMemcpyDeviceToHost);
    
    // Print the result
    std::cout << "Lower triangular matrix L:" << std::endl;
    for (int i = 0; i < n; i++) {
        for (int j = 0; j < n; j++) {
            std::cout << A[i*n + j] << " ";
        }
        std::cout << std::endl;
    }

    // Free the allocated GPU memory
    cudaFree(d_A);

    return 0;
}

'File written in /content/src/run.cu'

In [ ]:
!nvcc /content/src/run.cu -o run.exe
!./run.exe

/content/src/run.cu(18): warning #497-D: declaration of "n" hides template parameter

/content/src/run.cu(45): warning #497-D: declaration of "n" hides template parameter

/content/src/run.cu(75): error: no instance of function template "choleskyDecomposition_InPlace_Kernel" matches the argument list
            argument types are: (float *, int)

1 error detected in the compilation of "/content/src/run.cu".
Lower triangular matrix L:
2.44949 15 55 
6.12372 4.1833 225 
22.4537 20.9165 6.1101 


# Useful code from Brian

In [ ]:
template <typename T, unsigned VEC_SIZE, unsigned SMALL_SWITCH = 3>
__device__
void reducePlus(T *data, cgrps::thread_block block){
    unsigned size_left = VEC_SIZE;
    // loop until only a few values left
    while (size_left > SMALL_SWITCH){
        // determine if odd_adjust needed and update size
        bool odd_flag = size_left % 2;
        size_left = (size_left - odd_flag)/2; 
        // reduce in half
        for (unsigned ind = block.thread_rank(); ind < size_left; ind += block.num_threads()){
            data[ind] += data[ind + size_left];
        }	
        // add the odd size adjust if needed
        if (block.thread_rank() == 0 && odd_flag){data[0] += data[2*size_left];}
        // sync and repeat
        block.sync();
    }
    // when we get really small sum up what is left
    if (block.thread_rank() == 0){
        for(unsigned ind = 1; ind < size_left; ind++){data[0] += data[ind];}
    }
}

In [ ]:
/*******************************************************************************
*                            matrix inversion                                  *
*******************************************************************************/
    

// load identity in so memory is [A | I]
template <typename T, unsigned DIM>
__device__ __forceinline__
void loadIdentity(T *A){
    for (unsigned ind = GATO_THREAD_NUMBER; ind < DIM*DIM; ind += GATO_THREADS_PER_BLOCK){
        unsigned r, c;
        r = ind % DIM; 
        c = ind / DIM;
        A[ind] = static_cast<T>(r == c);
    }
}

// load identity in so memory is [V | I]
template <typename T, unsigned DIMA, unsigned DIMB>
__device__ __forceinline__
void loadIdentity(T *A, T *B){
    for (unsigned ind = GATO_THREAD_NUMBER; ind < DIMA*DIMA+DIMB*DIMB; ind += GATO_THREADS_PER_BLOCK){
        unsigned r, c, indAdj; T *V;
        if (ind < DIMA*DIMA){
            indAdj = ind;
            r = indAdj % DIMA; c = indAdj/DIMA; V = A;
        }
        else {
            indAdj = ind - DIMA*DIMA;
            r = indAdj % DIMB; c = indAdj/DIMB; V = B;
        }
        V[indAdj] = static_cast<T>(r == c);
    }
}


// load identity in so memory is [V | I]
template <typename T, unsigned DIMA, unsigned DIMB, unsigned DIMC>
__device__ __forceinline__
void loadIdentity(T *A, T *B, T *C){
    for (unsigned ind = GATO_THREAD_NUMBER; ind < DIMA*DIMA+DIMB*DIMB+DIMC*DIMC; ind += GATO_THREADS_PER_BLOCK){
        unsigned r, c, indAdj; T *V;
        if (ind < DIMA*DIMA){
            indAdj = ind;
            r = indAdj % DIMA; c = indAdj/DIMA; V = A;
        }
        else if (ind < DIMA*DIMA+DIMB*DIMB){
            indAdj = ind - DIMA*DIMA;
            r = indAdj % DIMB; c = indAdj/DIMB; V = B;
        }
        else{
            indAdj = ind - DIMA*DIMA - DIMB*DIMB;
            r = indAdj % DIMC; c = indAdj/DIMC; V = C;
        }
        V[indAdj] = static_cast<T>(r == c);
    }
}


template <typename T, unsigned DIM>
__device__
void invertMatrix(T *A, T *s_temp){ 
// we are going to guassian elimination walking down the matrix (assuming no leading 0s)
// we therefore use the columns in order as the pivot column for each pivot we need to rescale 
// that row so that the pivot value (pv) is 1 THEN for all other row values (orv) we need to add a multiple 
// of the NEW pivot row value (prv) such that we transorm the other row pivot column value (orpcv) to 0
// pr *= 1/pv   orv -= orpcv*prv == orv -= orpcv*1/pv*prvOld
    for (unsigned pivRC = 0; pivRC < DIM; pivRC++){
        unsigned pivColOffset = pivRC*DIM;
        // save the pivot and pivot column and row
        T pvInv = static_cast<T>(1)/A[pivRC + pivColOffset];
        for (unsigned ind = GATO_THREAD_NUMBER; ind < 2*DIM+1; ind++){
            unsigned AInd;
            if (ind < DIM){AInd = ind + pivColOffset;}
            else{AInd = pivRC + pivColOffset + (ind-DIM)*DIM;}
            s_temp[ind] = A[AInd];
        }
        __syncthreads(); //----------------------
        // make the pivot update
        for (unsigned ind = GATO_THREAD_NUMBER; ind < DIM*(DIM+1); ind += GATO_THREADS_PER_BLOCK){
            unsigned row = ind % DIM; unsigned col = ind / DIM; unsigned colOffset = ind - row;
            // s_temp = orpcvs|prvOld
            if (row == pivRC){A[row + pivColOffset + colOffset] *= pvInv;}
            else{A[row + pivColOffset + colOffset] -= s_temp[row]*pvInv*s_temp[DIM+col];}
        }
    __syncthreads(); //----------------------
    }
}


template <typename T, unsigned DIMA, unsigned DIMB, unsigned MAX_DIM>
__device__
void invertMatrix(T *A, T *B, T *s_temp){

    // now we are going to guassian elimination walking down the matrix (assuming no leading 0s)
    // we therefore use the columns in order as the pivot column for each pivot we need to rescale 
    // that row so that the pivot value (pv) is 1 THEN for all other row values (orv) we need to add a multiple 
    // of the NEW pivot row value (prv) such that we transorm the other row pivot column value (orpcv) to 0
    // pr *= 1/pv   orv -= orpcv*prv == orv -= orpcv*1/pv*prvOld
T *s_memA = s_temp; T *s_memB = &s_memA[2*DIMA+1];
    for (unsigned pivRC = 0; pivRC < MAX_DIM; pivRC++){
        bool AActive = pivRC < DIMA; bool BActive = pivRC < DIMB;
        unsigned pivColOffsetA = pivRC*DIMA; unsigned pivColOffsetB = pivRC*DIMB;
        // save the pivot column and row
        for (unsigned ind = GATO_THREAD_NUMBER; ind < MAX_DIM; ind++){
            if (AActive && ind < DIMA){s_memA[ind] = A[ind + pivColOffsetA];}
            if (BActive && ind < DIMB){s_memB[ind] = B[ind + pivColOffsetB];}
        }
        for (unsigned ind = GATO_THREAD_NUMBER; ind < MAX_DIM+1; ind++){
            if (AActive && ind < DIMA+1){s_memA[ind + DIMA] = A[ind*DIMA + pivRC + pivColOffsetA];}
            if (BActive && ind < DIMB+1){s_memB[ind + DIMB] = B[ind*DIMB + pivRC + pivColOffsetB];}
        }
        __syncthreads(); //----------------------
        // make the pivot update with s_mem = [colA,rowA,colB,rowB,colC,rowC]
        for (unsigned ind = GATO_THREAD_NUMBER; ind < MAX_DIM*(MAX_DIM+1); ind += GATO_THREADS_PER_BLOCK){
            if (AActive && ind < DIMA*(DIMA+1)){
                unsigned row = ind % DIMA; unsigned col = ind / DIMA;
                if (row == pivRC){A[pivColOffsetA + ind] /= s_memA[pivRC];}
                else{A[pivColOffsetA + ind] -= s_memA[row]/s_memA[pivRC]*s_memA[DIMA+col];}
            }
            if (BActive && ind < DIMB*(DIMB+1)){
                unsigned row = ind % DIMB; unsigned col = ind / DIMB; 
                if (row == pivRC){B[pivColOffsetB + ind] /= s_memB[pivRC];}
                else{B[pivColOffsetB + ind] -= s_memB[row]/s_memB[pivRC]*s_memB[DIMB+col];}
            }
        }
        __syncthreads(); //----------------------
    }
}

// invert A,B,C assume memory for all is [V | VInv] where both are DIMxDIM and continguous
// relies on s_temp of size [2*DIMA + 2*DIMB + 2*DIMC + 3]
template <typename T, unsigned DIMA, unsigned DIMB, unsigned DIMC, unsigned MAX_DIM>
__device__
void invertMatrix(T *A, T *B, T *C, T *s_temp){

    // now we are going to guassian elimination walking down the matrix (assuming no leading 0s)
    // we therefore use the columns in order as the pivot column for each pivot we need to rescale 
    // that row so that the pivot value (pv) is 1 THEN for all other row values (orv) we need to add a multiple 
    // of the NEW pivot row value (prv) such that we transorm the other row pivot column value (orpcv) to 0
    // pr *= 1/pv   orv -= orpcv*prv == orv -= orpcv*1/pv*prvOld
    T *s_memA = s_temp; T *s_memB = &s_memA[2*DIMA+1]; T *s_memC = &s_memB[2*DIMB+1];
    for (unsigned pivRC = 0; pivRC < MAX_DIM; pivRC++){
        bool AActive = pivRC < DIMA; bool BActive = pivRC < DIMB; bool CActive = pivRC < DIMC;
        unsigned pivColOffsetA = pivRC*DIMA; unsigned pivColOffsetB = pivRC*DIMB; unsigned pivColOffsetC = pivRC*DIMC;
        // save the pivot column and row
        for (unsigned ind = GATO_THREAD_NUMBER; ind < MAX_DIM; ind++){
            if (AActive && ind < DIMA){s_memA[ind] = A[ind + pivColOffsetA];}
            if (BActive && ind < DIMB){s_memB[ind] = B[ind + pivColOffsetB];}
            if (CActive && ind < DIMC){s_memC[ind] = C[ind + pivColOffsetC];}
        }
        for (unsigned ind = GATO_THREAD_NUMBER; ind < MAX_DIM+1; ind++){
            if (AActive && ind < DIMA+1){s_memA[ind + DIMA] = A[ind*DIMA + pivRC + pivColOffsetA];}
            if (BActive && ind < DIMB+1){s_memB[ind + DIMB] = B[ind*DIMB + pivRC + pivColOffsetB];}
            if (CActive && ind < DIMC+1){s_memC[ind + DIMC] = C[ind*DIMC + pivRC + pivColOffsetC];}
        }
        __syncthreads(); //----------------------
        // make the pivot update with s_mem = [colA,rowA,colB,rowB,colC,rowC]
        for (unsigned ind = GATO_THREAD_NUMBER; ind < MAX_DIM*(MAX_DIM+1); ind += GATO_THREADS_PER_BLOCK){
            if (AActive && ind < DIMA*(DIMA+1)){
                unsigned row = ind % DIMA; unsigned col = ind / DIMA;
                if (row == pivRC){A[pivColOffsetA + ind] /= s_memA[pivRC];}
                else{A[pivColOffsetA + ind] -= s_memA[row]/s_memA[pivRC]*s_memA[DIMA+col];}
            }
            if (BActive && ind < DIMB*(DIMB+1)){
                unsigned row = ind % DIMB; unsigned col = ind / DIMB; 
                if (row == pivRC){B[pivColOffsetB + ind] /= s_memB[pivRC];}
                else{B[pivColOffsetB + ind] -= s_memB[row]/s_memB[pivRC]*s_memB[DIMB+col];}
            }
            if (CActive && ind < DIMC*(DIMC+1)){
                unsigned row = ind % DIMC; unsigned col = ind / DIMC;
                if (row == pivRC){C[pivColOffsetC + ind] /= s_memC[pivRC];}
                else{C[pivColOffsetC + ind] -= s_memC[row]/s_memC[pivRC]*s_memC[DIMC+col];}
            }
        }
        __syncthreads(); //----------------------
    }
}

SyntaxError: ignored

In [ ]:
/**
     * Compute the dot product between two vectors
     *
     * Notes:
     *   Assumes computed by a single thread
     *
     * @param vec1 is the first vector of length N with stride S1
     * @param vec2 is the second vector of length N with stride S2
     * @return the resulting final value
     */
    template <typename T, int N, int S1, int S2>
    __device__
    T dot_prod(const T *vec1, const T *vec2) {
        T result = 0;
        for(int i = 0; i < N; i++) {
            result += vec1[i*S1] * vec2[i*S2];
        }
        return result;
    }

    /**
     * Compute the dot product between two vectors
     *
     * Notes:
     *   Assumes computed by a single thread
     *
     * @param vec1 is the first vector of length N with stride S1
     * @param vec2 is the second vector of length N with stride S2
     * @return the resulting final value
     */
    template <typename T, int N, int S1, int S2>
    __device__
    T dot_prod(T *vec1, const T *vec2) {
        T result = 0;
        for(int i = 0; i < N; i++) {
            result += vec1[i*S1] * vec2[i*S2];
        }
        return result;
    }

    /**
     * Compute the dot product between two vectors
     *
     * Notes:
     *   Assumes computed by a single thread
     *
     * @param vec1 is the first vector of length N with stride S1
     * @param vec2 is the second vector of length N with stride S2
     * @return the resulting final value
     */
    template <typename T, int N, int S1, int S2>
    __device__
    T dot_prod(const T *vec1, T *vec2) {
        T result = 0;
        for(int i = 0; i < N; i++) {
            result += vec1[i*S1] * vec2[i*S2];
        }
        return result;
    }

    /**
     * Compute the dot product between two vectors
     *
     * Notes:
     *   Assumes computed by a single thread
     *
     * @param vec1 is the first vector of length N with stride S1
     * @param vec2 is the second vector of length N with stride S2
     * @return the resulting final value
     */
    template <typename T, int N, int S1, int S2>
    __device__
    T dot_prod(T *vec1, T *vec2) {
        T result = 0;
        for(int i = 0; i < N; i++) {
            result += vec1[i*S1] * vec2[i*S2];
        }
        return result;
    }

# LowerTriBackSub - Tested, Works
Now from $Ax=b$  we have $LL^Tx=b$
 $$[
 L y = b
 ]$$
 for a lower-triangular matrix $L$, or
 $$[
  L^T x =y
 ]$$


In [ ]:
%%cuda -n lowerBackSub.cu

#include <stdio.h>
#include <iostream>
#include <cmath>

//Solves linear system of equations for a lower-trianguler matrix


template <typename T> 
__device__ 
void lowerBackSub_InPlace(T *s_A, T *s_b, bool istransposed, int n, int m) {
    for (unsigned col = 0; col < n; col++) {
        if (threadIdx.x == 0){
            for(unsigned col_l = 0 ; col_l < col; col_l++) {
                if (istransposed) {
                    s_b[col] -= s_A[col_l*n+col]; 
                }
                else {
                    s_b[col] -=s_A[col*n+col_l];
                }
            }
            s_b[col] /= s_A[col*n+col];            
        }
        __syncthreads(); //here we computed the b_col

        // multiply matrix by b_col
        for(unsigned row = threadIdx.x + col +1; row < n; row += blockDim.x){
           if (istransposed) {
              s_A[col*n+row] = s_A[col*n+row]*s_b[col];
           } else {
              s_A[row*n+col] = s_A[row*n+col]*s_b[col];
           }
        }
        __syncthreads();
    }
}
__device__
void SolveLeaf(T *s_A) {

     if(blockIdx.x == 0){
        lowerbacksub()
        
     }

     for (i =  blockIdx.x, i < n; i += gridDim.x){
        ddevice)_funct(i)
     }

}

template <typename T>
__global__ 
void lowerBackSub_InPlace_Kernel(T *d_A, T *d_b, bool istransposed, int n,int m) {
    __shared__ T s_A[9];
    __shared__ T s_b[3];

    extern __shared__ T s_temp[];
    T *s_A = s_temp;
    T *s_B = s_A + 

    // move RAM to shared
    for(unsigned i = threadIdx.x; i < n*n; i += blockDim.x)  {
        s_A[i] = d_A[i]; s_b[i] = d_b[i];}


    // compute the decomposition
    lowerBackSub_InPlace<T>(s_A, s_b, istransposed, n);

    // move shared to RAM
    for(unsigned i = threadIdx.x; i < n*n; i += blockDim.x) {
        d_A[i] = s_A[i]; d_b[i] = s_b[i];
    }
}



__host__
int main() {
    // Input matrix on the host
    int n=3;
    float A[n*n] = {1,1,1,0,1,1,0,0,1};
    float b[n] = {10,7,6};
    bool istransposed = true;


    // Allocate memory on the GPU for the input and output matrices
    float* d_A; cudaMalloc((void**)&d_A, n * n * sizeof(float));
    float* d_b; cudaMalloc((void**)&d_b, n*sizeof(int));

    // Copy the input matrix from the host to the GPU memory
    cudaMemcpy(d_A, A, n * n * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, b, n*sizeof(int), cudaMemcpyHostToDevice);

    //Launch the CUDA kernel with appropriate block and grid dimensions
    int blockSize = 256;
    int gridSize = 1;
    lowerBackSub_InPlace_Kernel<float><<<gridSize, blockSize, n>>>(d_A,d_b, istransposed);
    cudaDeviceSynchronize();

    // Copy the result back from the GPU memory to the host
    cudaMemcpy(A, d_A, n * n * sizeof(float), cudaMemcpyDeviceToHost);
    cudaMemcpy(b, d_b, n *sizeof(float), cudaMemcpyDeviceToHost);

    
    // Print the result
    std::cout << "solution vector b:" << std::endl;
    for (int i = 0; i < n; i++) {
        std::cout << b[i] << " ";
      
        std::cout << std::endl;
    }

    // Free the allocated GPU memory
    cudaFree(d_A); cudaFree(d_b);

    return 0;
}


'File written in /content/src/lowerBackSub.cu'

modified version that solves for $AX=B$

In [ ]:
%%cuda -n lowerBackSub.cu

#include <stdio.h>
#include <iostream>
#include <cmath>

//Solves linear system of equations for a lower-trianguler matrix


template <typename T> 
__device__ 

// A - nxn; X - nxm; B nxm;  
void lowerBackSub_InPlace(T *s_A, T *s_B, bool istransposed, int n, int m) {
    unsigned k = threadIdx.x;
    if (k < m) {
        if (istransposed) {
            for (int col = n-1; col >= 0; col--) {
                for(int col_l = n-1; col_l > col; col_l--) {
                    // s_B[k*n + col] -= s_A[col_l*n+col] * s_B[k*n + col_l]; by right it should be this but because the matrix we put in the the lower
                     s_B[k*n + col] -= s_A[col_l*n+col] * s_B[k*n + col_l];
                }
                // s_B[k*n + col] /= s_A[col*n+col];
            }
        } else {
            for (unsigned col = 0; col < n; col++) {
                for(unsigned col_l = 0 ; col_l < col; col_l++) {
                    s_B[k*n + col] -= s_A[col_l*n+col] * s_B[k*n + col_l];
                }
                s_B[k*n + col] /= s_A[col*n+col];
            }
        }
    }
    __syncthreads(); // here we computed the b_col
}


template <typename T>
__global__ 
void lowerBackSub_InPlace_Kernel(T *d_A, T *d_B, bool istransposed, int n, int m) {
    __shared__ T s_A[9]; // maybe this is okay to hardcode since the matrices put in are definitely this size? not sure
    __shared__ T s_B[6]; // what should the dimensions of this be? maybe make it 3x2 for the timebeing?

    // move RAM to shared
    // does this mean that all of the threads are doing this?
    // fast way of moving all this into RAM - each thread responsible for moving the data
    for(unsigned i = threadIdx.x; i < n*n; i += blockDim.x)  {
        s_A[i] = d_A[i]; s_B[i] = d_B[i];}


    // compute the decomposition
    lowerBackSub_InPlace<T>(s_A, s_B, istransposed, n, m);

    // move shared to RAM
    for(unsigned i = threadIdx.x; i < n*n; i += blockDim.x) {
        d_A[i] = s_A[i];
    }
    for(unsigned i = threadIdx.x; i < n*m; i += blockDim.x) {
        d_B[i] = s_B[i];
    }
}



__host__
int main() {
    // Input matrix on the host
    int n=3;
    int m=2;
    float A[n*n] = {2,0,0,1,1,0,3,3,3};
    float B[n*m] = {10,7,6,10,7,6};
    bool istransposed = true;


    // Allocate memory on the GPU for the input and output matrices
    float* d_A; cudaMalloc((void**)&d_A, n * n * sizeof(float));
    float* d_B; cudaMalloc((void**)&d_B, n * m * sizeof(int));

    // Copy the input matrix from the host to the GPU memory
    cudaMemcpy(d_A, A, n * n * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, B, n * m * sizeof(int), cudaMemcpyHostToDevice);

    //Launch the CUDA kernel with appropriate block and grid dimensions
    int blockSize = 256;
    int gridSize = 1;
    lowerBackSub_InPlace_Kernel<float><<<gridSize, blockSize>>>(d_A,d_B, istransposed, n, m);
    cudaDeviceSynchronize();

    // Copy the result back from the GPU memory to the host
    cudaMemcpy(A, d_A, n * n * sizeof(float), cudaMemcpyDeviceToHost);
    cudaMemcpy(B, d_B, n * m * sizeof(float), cudaMemcpyDeviceToHost);

    
    // Print the result
    std::cout << "solution matrix X:" << std::endl;
    for (int i = 0; i < n; i++) {
        for (int j = 0; j < m; j++) {
            std::cout << B[j*n+i] << " ";
        }
        std::cout << std::endl;
    }

    // Free the allocated GPU memory
    cudaFree(d_A); cudaFree(d_B);

    return 0;
}


'File written in /content/src/lowerBackSub.cu'

In [ ]:
cudaDevicesybchronize - synch the kernel


In [ ]:
!nvcc /content/src/lowerBackSub.cu -o lowerBackSub.exe
!./lowerBackSub.exe

solution matrix X:
5 5 
7 7 
2 2 


# Cholesky Solve - Not tested

Solves a linear system of equation with a precomputed Cholesky decomposition.

 * @param[in]    A A square matrix whose Cholesky decomposition is stored in the lower triangular portion of the matrix
 * @param[inout] b The right-hand-side vector. Stores the solution upon completion of the function.
 * @return 0 if successful

In [ ]:
%%cuda -n CholeskySolve.cu

#include <stdio.h>
#include <iostream>
#include <cmath>

//Solves linear system of equations with Cholesky
template <typename T> 
__device__ 
void cholSolve_InPlace(T *s_A, T *s_b, bool istransposed, int n, int m) {
    lowerBackSub_InPlace<T>(s_A, s_b, 0, n, m);
    lowerBackSub_InPlace<T>(s_A, s_b, 1, n, m);
}


template <typename T>
__global__ 
void CholSolve_InPlace_Kernel(T *d_A, T *d_b, int n) {
    __shared__ T s_A[9];
    __shared__ T s_b[3];

    // move RAM to shared
    for(unsigned i = threadIdx.x; i < n*n; i += blockDim.x)  {
        s_A[i] = d_A[i]; s_b[i] = d_b[i];}


    CholSolve_InPlace<T>(s_A, s_b, false, n);

    // move shared to RAM
    for(unsigned i = threadIdx.x; i < n*n; i += blockDim.x) {
        d_A[i] = s_A[i]; d_b[i] = s_b[i];
    }
}



__host__
int main() {
    // Input matrix on the host
    int n=3;
    float A[n*n] = {2,0,1,1,2,3,1,0,3};
    float b[n] = {6,7,6};
    bool istransposed = false;


    // Allocate memory on the GPU for the input and output matrices
    float* d_A; cudaMalloc((void**)&d_A, n * n * sizeof(float));
    float* d_b; cudaMalloc((void**)&d_b, n*sizeof(int));

    // Copy the input matrix from the host to the GPU memory
    cudaMemcpy(d_A, A, n * n * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, b, n*sizeof(int), cudaMemcpyHostToDevice);

    //Launch the CUDA kernel with appropriate block and grid dimensions
    int blockSize = 256;
    int gridSize = 1;
    CholSolve_InPlace_Kernel<float><<<gridSize, blockSize>>>(d_A,d_b, istransposed, n);
    cudaDeviceSynchronize();

    // Copy the result back from the GPU memory to the host
    cudaMemcpy(A, d_A, n * n * sizeof(float), cudaMemcpyDeviceToHost);
    cudaMemcpy(b, d_b, n *sizeof(float), cudaMemcpyDeviceToHost);

    
    // Print the result
    std::cout << "solution vector b:" << std::endl;
    for (int i = 0; i < n; i++) {
        std::cout << b[i] << " ";
      
        std::cout << std::endl;
    }

    // Free the allocated GPU memory
    cudaFree(d_A); cudaFree(d_b);

    return 0;
}

'File written in /content/src/CholeskySolve.cu'


#GLASS


In [ ]:
//L1 - vector by vector/scalar
#include axpy.cuh //multiply vector x*alp+y
#include copy.cuh // copy vector x to y
#include dot.cuh // dot product between x and y vectors
#include ident.cuh // load identity next to matrix
#inclide scal.cuh // multiply vector by scalar
#include swap.cuh //swap x and y vectors
//L2 - matrix by vector
#include gemv.cuh //just matrix by vector?
#include gemm.cuh //NOT CHECKED
#include inv.cuh //invert matrix

#**Memory Explanation**


linear index = index+ nhorizon*level

**C factor** is usually used as solver->data \\
>**C.state = A matrix**, nstates^2 \\
to access C.state use 
A_B[k * dyn_step] where k is a time step or linear index

>**C.input = B matrix** , nstates*ninputs \\
to access C.input use A_B[k * dyn_step+(nstates*nstates] where k is a linear index\\
 
 >C.lambda - UNUSED PORTION OF THE NDFACTOR


 **zFactor is usually used as solver -> soln**\\
 >**z.state = q vector** of nstates \\
 to access z.state use q_r[k * (nstates+ninputs)] \\

 >**z.input = r** vector of size ninputs \\
 to access z.input use q_r[k * (nstates+ninputs)+nstates]

 >**z.lambda = $x_0$+d , total vector of size nstates** \\
 to access z.lambda use d[k *  nstates] \\

Q and R matrices are stored in diagonals array in Brian's code and in *Q_R in ours.
 >**diagonals[2k] stands for Q matrix**, nstates*nstates \\
 to access use Q_R[k * cost_step]

 >**diagonals[2k+1] stands for R matrix**, ninputs*ninputs \\
 to access use Q_R[k*coststep+(nstates*nstates)]


**Need to come up with array structure for F**.
>F.state = matrix of nstates^2 \\
F.input = matrix of ninput*nstates \\
F.lambda = matrix of nstates^2 \\

DURING/AFTER SOLVE LEAF
>F.lambda stores results of C.state*-1
>F.state stores 0.0 for k=0, and then stores MatrixCholeskySolveWithInfo(Q, &C->state,
                                  Qchol)  

>F.input stores results of cholSolve(R,&C->input),at the last step initializes it with 0.0

                  





In [ ]:
/*Our implementation of memory layout, all matrices should be entered 
column major order*/

 if we want to save space to have Q_R as vectors:
//int cost_step = nstates+ninputs; //size of one timestep in array

//or if needed to have Q_R as matrices:
int cost_step = nstates*nstates+ninputs*ninputs; //size of one timestep in array
double* Q_R = (double*)malloc(nhorizon*cost_step*sizeof(double)); //-ninputs
double* q_r = (double*)malloc((nstates+ninputs)*nhorizon*sizeof(double)); //-ninputs

int dyn_step = nstates*nstates+ninputs*ninputs; //size of one timestep in array
double* A_B = (double*)malloc(nhorizon*dyn_step*sizeof(double));
//A,B should be entered as transposed matrices to A_B
double* c = (double*)malloc(nhorizon*sizeof(double));
double* d = (double*)malloc(nstates*nhorizon*sizeof(double));


//For factorized infor prob want
double* F_Qsolv;
double* F_Rsolv;
double* F_lambda; //??



//to go from Brian's tree to linear index
lin_index = index+ nhorizon*level;

//Figure out the size of the array for F.state+F.input, not sure if we need it
double* Fst_inp = (double*)malloc(nhorizon*)

In [ ]:
//From Brian's paper
int cost_size = 2*nstates + 2*ninputs+1; // Q,R,q,r,c

int dynamics_size = nstates*nstates +nstates*ninputs +nstates; //A.B.d
//int total_size = cost_size+dynamics_size; //in case we'll want have one array later one

double* data_cost = (double*)malloc(total_size*sizeof(double));
double* Q = data;
double* R = data+nstates; //skips over nstates (which is Q)
double* q = data + nstates + ninputs; //skips over Q,R
double* r = data + 2*nstates +ninputs; //r is size ninput
double* c = data+ 2 * nstates +2 * ninputs; 
double* A = data+ cost_size ; 
double* B = data+cost_size+ nstates*nstates;
double * d = data+cost_size+nstates*nstates+nstates*ninputs;

# **Solve Leaf**

In [ ]:
//GetNdFactor(nddata,index,level, &factor)
#include "./GLASS/GLASS.cuh"
#include <cstdint>
#include <cooperative_groups.h>
namespace cgrps = cooperative_groups;


__device__
void solveLeaf(int index, int nstates, int ninputs, int nhorizon, T *Q_R, T *q_r, T *A_B,
               T *c,T* d, cgrps::thread_group g = cgrps::this_thread_block()) {

  double* F_lambda = NULL; //can we just use vectors?
  double* F_state; //not sure if we need it?
  double* F_input; //not sure if we need it - substitute with an array F?
  double* Qchol = NULL;
  double* Rchol = NULL;
  double* zy_temp; // not
  double* Q;
  double* R;

  int k = index;
  
  //initial timestep
  if(index == 0) {
        Q = &Q_R[const_step*k]; ///or just Q_R ?
        R = &Q_R[const_step*k+nstates*nstates];

        //Solve the block system of equations.
        glas::copy(nstates*nstates,A_B[k*dyn_step],F_lambda);
        glas::scal(nstates*nstates,-1.0, F_lambda); 
        glas::const(nstates*nstates,F_state, 0);
        glas::copy(nstates*ninputs,A_B[k*(nstates*nstates)],F_input);    
        glass:chol_InPlace(ninputs, R);
        cholSolve_InPlace(R, F.input, ninputs,nstates); //ASK XIAN! Fu = R\Cu
        cholSolve_InPlace(R, q_r[k*(nstates+ninputs)+nstates], ninputs, 1); 
        //zu = R\zu

        //Solve the block system of eqn overwriting the rhs vector
        glas::copy(nstates,d[k*(nstates)],zy_temp); //MatrixCopy(&zy_temp, &z->lambda)
        glas::copy(nstates,q_r[k*(nstates+ninputs)], d[k*(nstates)]);
        glas::gemv(nstates,nstates,-1.0, Q, zy_temp, -1.0, d[k*(nstates)]);
        // zy = - Q * zy - zx
        glass:copy(zy_temp,q_r[k*(nstates+ninputs)]);
        glass::scal(nstates,-1.0,q_r[k*(nstates+ninputs)]); // zx = -zy
        glass:chol_InPlace(nstates,Q) //not sure if that's ok to have it inplace
    
      } else {
          
        int level = 0;
        Q = &Q_R[const_step*k]; 
        glass:chol_InPlace(nstates,Q);
        
        //Not the last timestep
        if(k<nhorizon -1) {
            //what to do with level? do I just continue to use k?
            R = &Q_R[const_step*k+nstaes*nstates];
            glass:chol_InPlace(ninputs,R);
            cholSolve_InPlace(R, q_r[k*(nstates+ninputs)+nstates],ninputs,1);
            //zu = R \ zu 

            glass::copy(A_B[k*dyn_step],F_state);
            cholSolve_InPlace(Q,F_state,nstates,nstates);

            glass::copy(A_B[k*dyn_step+(nstates*nstates)],F_input);
            cholSolve_InPlace(R,F_input); 
        }

        //Only the last timestep
        cholSolve_InPlace(Q,q_r[k*(nstates+ninputs)]);
        //int prev_level = ndlqr_GetIndexLevel(&solver->tree,k-1); ??
         // NOTE: This is -I on the state for explicit integration
          //For implicit integrators we'd use the A2, B2 partials wrt the next
          //state and control
        //update k
        glas::copy(A_B[k*dyn_step],F_state); // should be -I matrix, chekc manually!
        cholSolve_InPlace(Q,F_state);
        glas::const(nstates*nstates,F_state, 0.0);
        ;
        }
      
  }

# **Factor Inner Product**

In [ ]:
//fact_state: Factorization of Q\A - Do this within function or assume the factorization is passed in as the parameter?
//fact_input: Factorization of R\B
//fact_lambda: -A

//data_state: A
//data_input: B

__device__
double* FactorInnerProduct(T* A_B, T* fact_state, T* fact_input, T* fact_lambda, int index, int data_level, int fact_level, int nstates, int ninput, int nhorizon) {
    //dyn_step: nstates*nstates+ninputs*ninputs;
    //k: index + nhorizon*level

    double* C1_state; //A, A_B[k * dyn_step] where k is a time step or linear index
    double* C1_input; //B, A_B[k * dyn_step+nstates*nstates] where k is a linear index

    double* F1_state;
    double* F1_input;
    double* F1_lambda;

    double* C2_state; //A, A_B[k * dyn_step] where k is a time step or linear index
    double* C2_input; //B, A_B[k * dyn_step+nstates*nstates] where k is a linear index

    double* F2_state;
    double* F2_input;
    double* F2_lambda;

    //GetNdFactor: Retrieve individual NdFactor out of NdData
    int linear_index = index + nhorizon * data_level;
    C1_state = A_B+(linear_index * dyn_step);
    C1_input = A_B+(linear_index * dyn_step + nstates * nstates);

    linear_index = index + nhorizon * fact_level; //Not sure
    F1_state = fact_state+linear_index;
    F1_input = fact_input+linear_index;

    linear_index = (index + 1) + nhorizon * data_level;
    C2_state = A_B+(linear_index * dyn_step);
    C2_input = A_B+(linear_index * dyn_step + nstates * nstates);

    linear_index = (index + 1) + nhorizon * fact_level;
    F2_state = fact_state+linear_index;
    F2_input = fact_input+linear_index;

    //Use the precomputed Cholesky factorization to solve for y at each parent level 
    double *S = F2_lambda;
    glass::gemm(nstates, nstates, ninput, 1.0, C1_state, F1_state, -1.0, S); //S = C1x'F1x, why -1.0 and not 0.0?
    glass::gemm(nstates, ninput, nstates, 1.0, C1_input, F1_input, 1.0, S);
    glass::gemm(nstates, nstates, nstates, 1.0, C2_state, F2_state, 1.0, S);
    glass::gemm(nstates, ninput, nstates, C2_input, F2_input, 1.0, S);

    return S;
}

# SolveCholeskyFactor

In [ ]:
%%cuda -n lowerBackSub.cu

#include <stdio.h>
#include <iostream>
#include <cmath>

template <typename T> 
__device__ 
void SolveCholeskyFactor(T* fact_state, T* fact_input, T* fact_lambda, int index, int level, int upper_level, int nstates, int ninput, int nhorizon) {
    T *s_A = fact_lambda+123; // find what Sbar from level
    T *s_b = fact_lambda+123; // find f from upper_level
    cholSolve_InPlace(T *s_A, T *s_b, 0, nstates, 1);
}

#**Update Shur Factor**

In [ ]:
__device__
void UpdateShurFactor() {
    
}

In [ ]:
%%cuda diag_Matrix_set.cu

#include <cstdint>
#include <cooperative_groups.h>
namespace cgrps = cooperative_groups;

template <typename T>
__device__
void diag_Matrix_set(T *v_Q, T *m_Q, int n, cgrps::thread_group g) {
    
    for(uint32_t ind = g.thread_rank(); ind < n; ind+= g.size()){
        m_Q[ind*n+ind] = v_Q[n];        
    }
    
}